# Introduction to Image Processing and Analysis with Python  
   
This notebook provides a comprehensive guide to processing and analyzing images using Python. It leverages various libraries and tools to fetch, manipulate, and analyze images, particularly focusing on extracting and deduplicating image data, as well as integrating with cloud services for data storage and processing.  
   
## Key Libraries and Tools  
   
1. **Python Libraries**:  
   - `requests`: For fetching images from URLs.  
   - `base64`: For encoding image content.  
   - `pandas`: For data manipulation and analysis.  
   - `PIL` (Python Imaging Library): For image processing.  
   - `hashlib`: For computing image hashes.  
   - `tqdm`: For progress bars.  
   - `dotenv`: For loading environment variables.  
   - `google.cloud.bigquery`: For interacting with Google BigQuery.  
   
2. **LangChain**:  
   - `AzureChatOpenAI`: For integrating with Azure OpenAI services.  
   - `BaseModel` and `Field` from `langchain_core.pydantic_v1`: For creating structured data models.  
   - `HumanMessage` and `chain` from `langchain_core.messages` and `runnables`: For creating and running message chains.  
   - `JsonOutputParser`: For parsing JSON outputs.  

      
## Notebook Workflow  
   
### 1. Setup and Initialization  
The notebook begins by importing necessary libraries and setting up environment variables using `dotenv`. It also configures pandas display options for better readability.  

In [1]:
import sys
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
load_dotenv(".env", override=True)
import base64
import requests
import pandas as pd

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage
from typing import Literal, List, Union, Optional
from langchain_core.runnables import chain
from langchain_core.output_parsers import JsonOutputParser
from google.cloud import bigquery
import tqdm

from PIL import Image  
from io import BytesIO  
from IPython import display
from IPython.display import Markdown, Video, display
import cv2
import numpy as np

import hashlib  
from tqdm import tqdm 
from google.cloud import storage
from PIL import Image  # Using PIL for image manipulation

from tools.octocloud import Dataframe, Table, Directory
import time

from pydantic import BaseModel, Field, constr, field_validator  
from typing_extensions import Literal

# Pandas display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)  

In [ ]:
SAMPLE_SIZE = 400

# Constants
client = bigquery.Client(os.getenv("PROJECT_ID"))
directory = Directory(client, os.getenv("PROJECT_ID"), dataset="renault_crea")
input_ads_table = Table(client, directory, "fb_ads_test")
intermediate_image_table = Table(client, directory, "fb_ads_image_urls_intermediate")
intermediate_video_table = Table(client, directory, "fb_ads_video_urls_intermediate")
output_image_table = Table(client, directory, "fb_ads_image_final")
output_video_table = Table(client, directory, "fb_ads_video_final")

# Clean tables used for dashboard
features_image_table = Table(client, directory, "fb_ads_image_features")
features_video_table = Table(client, directory, "fb_ads_video_features")

### 2. Data import


In [ ]:
sql_image = """ 
SELECT
  *
FROM(
  SELECT 
    id, 
    category,
    image_url, 
  FROM {input_table} LEFT JOIN UNNEST(image_urls) AS image_url
)
WHERE image_url IS NOT NULL
"""

sql_video = """ 
SELECT
  *
FROM(
  SELECT 
    id, 
    category,
    video_url, 
  FROM {input_table} LEFT JOIN UNNEST(video_urls) AS video_url 
)
WHERE video_url IS NOT NULL
"""



query_image = sql_image.format(input_table=input_ads_table.path("standard"))
query_video = sql_video.format(input_table=input_ads_table.path("standard"))
df_image = client.query_and_wait(query_image).to_dataframe()
df_video = client.query_and_wait(query_video).to_dataframe()
print("Number of images:", len(df_image))
print("Number of images:", len(df_video))


In [4]:
# Reduce the size of the sample in test phase
df_test_image = df_image.head(SAMPLE_SIZE).copy()
df_test_video = df_video.head(round(SAMPLE_SIZE/4)).copy()

### 3. Image processing
#### Data cleaning: Image deduplication and perceptual hashing

Hashing is the transformation of any data into a usually shorter fixed-length value or key that represents the original string.

Just as we have unique Fingerprints, Hashes are unique for any particular data. There are lots of Hashing Algorithms out there which cater to specific needs.

Perceptual hash is like a regular hash in that it is a smaller, compare-able fingerprint of a much larger piece of data. However, unlike a typical hashing algorithm, the idea with a perceptual hash is that the perceptual hashes are “close” (or equal) if the original images are close.

In [4]:
def dhash(image, hash_size=8):
    """
    Generates a dHash for the input image.
    Args:
        image (PIL.Image.Image): Input image
        hash_size: the size to which the image has to be resized for hash calculation
    Returns:
        str: Hexadecimal representation of the hash.
    """
    resized = image.resize((hash_size + 1, hash_size), Image.LANCZOS).convert('L')
    difference = []
    for row in range(hash_size):
        for col in range(hash_size):
            pixel_left = resized.getpixel((col, row))
            pixel_right = resized.getpixel((col + 1, row))
            difference.append(pixel_left > pixel_right)

    # Convert boolean list to integer, then to hex representation
    decimal_value = int("".join([str(int(b)) for b in difference]), 2)
    hash_hex = hex(decimal_value)[2:]
    
    # If the hex string has less digits, left pad with zeros
    hash_hex_len = hash_size * hash_size / 4 # Divide by 4 to get correct no of hex characters as 4 bits can be represented using 1 hex character.
    hash_hex =  hash_hex.zfill(int(hash_hex_len))
    
    return hash_hex

def image_to_bytes(gcs_url):
    """Loads an image from Google Cloud Storage and returns a PIL Image object.
    Args:
        gcs_url (str): Full GCS URL of the image (e.g., "gs://bucket/image.jpg").
    Returns:
        PIL.Image.Image: PIL Image object, or None if an error occurred.
    """
    try:
        bucket_name, blob_name = gcs_url.replace("gs://", "").split("/")[-2:]
        client = storage.Client(os.getenv("PROJECT_ID"))
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        image_bytes = blob.download_as_bytes()
        return image_bytes
    except Exception as e:
        print(f"Error loading image from GCS: {e}")
        return None

def image_to_pillow(gcs_url):

    image = Image.open(BytesIO(image_to_bytes(gcs_url)))
    return image

    
def calculate_aspect(width: int, height: int) -> str:
    def gcd(a, b):
        """The GCD (greatest common divisor) is the highest number that evenly divides both width and height."""
        return a if b == 0 else gcd(b, a % b)

    r = gcd(width, height)
    x = int(width / r)
    y = int(height / r)

    return f"{x}:{y}"

In [ ]:
# Iterate over each row in the DataFrame, showing a progress bar  
for index, row in tqdm(df_test_image.iterrows(), total=df_test_image.shape[0]):  
    gcs_url = row["image_url"]  # Retrieve the image URL from the "image_url" column of the current row  
    image = image_to_pillow(gcs_url)  # Convert the image URL to a Pillow Image object  
  
    if image:  
        # If the image was successfully loaded, update the corresponding columns in the DataFrame  
        df_test_image.loc[index, "image_hash"] = dhash(image)  # Calculate the image hash and store it in the "image_hash" column  
        df_test_image.loc[index, "size"] = f"{image.size[1]}:{image.size[0]}"  # Store the image size in the "size" column  
        df_test_image.loc[index, "aspect_ratio"] = calculate_aspect(image.size[1], image.size[0])  # Calculate the aspect ratio and store it in the "aspect_ratio" column  
    else:  
        # If the image could not be loaded, update the corresponding columns with None values  
        df_test_image.loc[index, "image_hash"] = None  
        df_test_image.loc[index, "size"] = None  
        df_test_image.loc[index, "aspect_ratio"] = None  

In [ ]:
# Check the content of one image

image_to_pillow(gcs_url)

#### Export of intermediate table to BigQuery

In [ ]:
df_test_image.reset_index(inplace=True)
Dataframe(client, df_test_image).to_table(intermediate_image_table)

#### Feature extraction from image

1. First we retrieve the unique images from the intermediary table (after image deduplication)
2. Initialize a GenAI client, which is used to interact with the Gemini 2.0 Flash Experimental model.  
3. Define a data model called `ImageInformation` using Pydantic. This model represents the features that will be extracted from each image.  
4. Create an empty list called `features` to store the extracted features for each image.  
5. Iterating over the dataframe, we make a request to the Gemini 2.0 Flash Experimental model to generate content based on the image.  
6. We retrieve the response from the model is parsed as JSON and stored in the `feature_dict` variable.  
7. The `feature_dict` is updated with additional information such as the image URL and image hash to join it later onto the original table.
11. We export the final list of feature dict to a BigQuery table using the octocloud script

In [ ]:
# SQL query to retrieve data from the intermediate_image_table  
sql = """   
SELECT     
  image_hash,  
  MAX(image_url) AS image_url,
FROM {intermediate_image_table}  
GROUP BY 1
"""  
  
# Format the SQL query with the table name  
query = sql.format(  
    intermediate_image_table=intermediate_image_table.path("standard")  
)  
  
# Execute the query and wait for the result, then convert it to a DataFrame  
df_deduplicated_images = client.query_and_wait(query).to_dataframe()  
  
# Print the number of images in the DataFrame  
print("Number of images:", len(df_deduplicated_images))  

In [ ]:
from google import genai
from google.genai import types
from pydantic import BaseModel, Field, constr, field_validator  
from typing import List, Union
from typing_extensions import Literal
from typing import Optional
import json 

genai_client = genai.Client(
    vertexai=True, project=os.getenv("PROJECT_ID"), location='us-central1'
)



class ImageInformation(BaseModel):  
    """Information about an image."""  
      
    image_description: str = Field(description="A brief one-sentence description of the creative.")  
    product_type: Literal["car", "service", "connected services", "accessories"] = Field(description="The type of product represented by the advertisement.")  
    contrast_presence: bool = Field(description="Is the contrast strong between text and background color? True if 'yes', False if 'no'.")  
    product_elements: List[str] = Field(description="List of short descriptions of products in the creative.")
    # meta_text: bool = Field(description="Is the text displayed in the first 5 seconds of the video? True if 'yes', False if 'no'.") 
    # meta_video_length: bool = Field(description="Is the video duration more than 15 seconds ? True if 'yes', False if 'no'.") 
    # meta_product_presence: bool = Field(description="Is the product (car) displayed in the first 3 seconds of the video? True if 'yes', False if 'no'.") 
    product_presence: bool = Field(description="Is the car displayed in the ad? True if 'yes', False if 'no'.")  
    # product_share: int = Field(default = None, description="Percentage of video duration where the product is displayed")  
    product_percent: int = Field(description="Percentage of the creative area covered by the product.")  
    main_objects: List[str] = Field(description="List of the main objects in the picture.")  
    text: bool = Field(description="Indicates if there is text in the creative. True if 'yes', False if 'no'.")  
    text_location: Literal["top", "down", "right", "left", "center", "None"]= Field(default="None", description="Locations ad related text that is not the call to action")  
    brand_logo: bool = Field(description="Indicates if there is a brand logo on the car. True if 'yes', False if 'no'.")  
    promotion: bool = Field(description="Indicates if there is a promotion call in the creative. True if 'yes', False if 'no'.")  
    promotion_deadline: bool = Field(description="Indicates if a promotion deadline is displayed in the creative. True if 'yes', False if 'no'.")  
    promotion_theme: str= Field(default= "None", description="Specifies if the promotion is linked to a special event (e.g., Black Friday, Christmas).")  
    call_to_action: bool = Field(description="Indicates if there is a short call to action in the creative. The call to action should include a verb and it's length should have a maximum of 4 words. True if 'yes', False if 'no'.")  
    call_to_action_size: int = Field(default=None, description="Percentage of the creative area covered by the call to action, if applicable.")  
    car_brand: str = Field(default="None", description="The brand of the car, only if it is displayed on the product itself.")  
    car_model: str = Field(default="None", description="The model of the car, if applicable.")  
    price: bool = Field(description="Indicates if a price is displayed in the creative. True if 'yes', False if 'no'.")  
    ad_purpose: str = Field(default="awareness", description="The intended action promoted by the verb mentionned in the text (e.g., buy, order, discover, test).")  
    text_extraction: str = Field(default="None", description="The text content extracted from the creative.")  
    price_size: int = Field(default="None", description="Percentage of the creative area covered by the price, if applicable.") 
    price_type: Literal["None", "net price", "recurring monthly"] = Field(default="None", description="The format in which the price is presented. This can either be a one-time full net price or a recurring monthly price.") 
    color_palette: List[str] = Field(description="Dominant colors used in the creative.")  
    human_elements: List[str] = Field(default="None", description="Short descriptions of main humans in the creative.")  
    human_presence: bool = Field(description="Are there any distinguishable humans displayed in the ad? True if 'yes', False if 'no'.")  
    human_number: int = Field(default="None", description="Number of distinguishable humans in the creative.")  
    human_genders: List[str] = Field(default="None", description="List of humans' gender in the picture.")  
    imagery_type: Literal["photographs", "illustration", "mix media", "infographics", "3D rendering", "icons"] = Field(description="Type of imagery in the creative.")  
    style: List[str] = Field(description="General aesthetic and style of the creative (e.g., modern, vintage, minimalist, abstract).")  
    actions: List[str] = Field(description="Activities or actions taking place in the creative (e.g., running, jumping, eating).")  
    actions_presence: bool = Field(description="Is there an action/story taking place in the ad? True if 'yes', False if 'no'.")  
    environment: List[Literal["indoor", "outdoor", "urban", "rural", "None"]] = Field(description="The environment in which the scene is set.")  
    time_of_day: Literal["not identifiable", "daytime", "nighttime", "sunset", "dawn"] = Field(description="The time of day in the creative.")  
    weather: Literal["None", "sunny", "cloudy", "rainy"] = Field(default="None", description="The weather condition in the creative, if identifiable.")  
    background_main_color: str = Field(default="None", description="Dominant color of the background.")  
    human_shot_size: List[str] = Field(default="None", description="Shot size of the human in the creative (e.g., full shot, medium full shot, cowboy shot, closeup).")  
    human_age: Literal["senior", "adult", "child", "baby"] = Field(default="None", description="The age group of the human(s) in the  if they are distinguishable.")
    electric_vehicle: bool = Field(description="Is the ad related to the electrification of the vehicle? True if 'yes', False if 'no'.")  
    goal_awareness_or_consideration_or_conversion: Literal["awareness", "consideration", "conversion"] = Field(default="None", description="The ad's purpose based on the call to action, estimation is linked to consideration, while configuration is linked to consideration.") 
    # meta_logo_display: bool = Field(description="Is the brand referenced in the first 3 seconds of the video, without considering the logo on the product? True if 'yes', False if 'no'.")
    # branding_share: Optional[int] = Field(None, description="Percentage of the video where Renault branding (logo, text, etc.) is shown.")  
    black_bars: bool = Field(description="Is there a black bar on the creative? True if 'yes', False if 'no'.")  
    text_length: int = Field(default = None, description="Number of lines the text takes on the creative without considering the call to action.") 
    promotion_display: bool = Field(description="Indicates whether a price discount, or price promotion message is displayed in the ad. True if 'yes', False if 'no'.")  
    call_to_action_verb: str = Field(default="None", description="The specific verb used in the call to action, if present (e.g., 'buy', 'test').")  
    promotion_wording_type: Literal["None", "gain", "loss_aversion"] = Field(default="None", description="Specifies the type of promotion wording used in the ad: 'gain' for emphasizing benefits, 'loss_aversion' for emphasizing avoidance of loss, or 'None' if there is no promotion wording.")  
    persuasion: Literal["None", "informative", "persuasive", "emotional", "humorous", "inspirational", "authoritative", "urgent", "relatable"] = Field(default="None", description="The tone of voice used for the advertisement, if applicable (e.g., Humour, Surprise, Suspense, Love, Happiness, Neutral).")  
    new_old_vehicle: Literal["None", "new", "second-hand"] = Field(default="None", description="Indicates whether the advertised vehicle is new or old.")  
    # audio_presence: Optional[bool] = Field(None, description="Is there audio (music, voice, ...) in the video? True if 'yes', False if 'no'.")  
    # subtitles_presence: Optional[bool] = Field(None, description="Are there subtitles in the video? True if 'yes', False if 'no'.")  
    product_usage: bool = Field(description="Is the product (car, connected feature, etc.) used in the Ad? Is there a creative of someone driving the car? True if 'yes', False if 'no'.")  
    # action_audio: bool = Field(description="Is there a clear audio voice that is delivering the call to action? True if 'yes', False if 'no'.") 

def retry(fun, max_tries=3, sleep=1):
    for i in range(max_tries):
        try:
           fun()
           break
        except Exception as e:
            print(e)
            time.sleep(sleep) 
            continue

features_image = []


def append_features_image():

    # Load the image from GCS
    gcs_url = row["image_url"]
    bytes = image_to_bytes(gcs_url)

    response = genai_client.models.generate_content(
        model='gemini-2.0-flash-exp', 
        contents=[
            'Describe the content of the picture',
            types.Part.from_bytes(data=bytes, mime_type="image/png")    
        ],
        config=types.GenerateContentConfig(
            temperature=0,
            response_mime_type= 'application/json',
            response_schema= ImageInformation
        )
    )

    feature_dict = json.loads(response.text)
    feature_dict.update({"image_url": row["image_url"], "image_hash": row["image_hash"]})
    features_image.append(feature_dict)
    return response

for index, row  in tqdm(df_deduplicated_images.iterrows(), total=df_deduplicated_images.shape[0]):
    response = retry(append_features_image)


#### Features post processing

Cleaning of the extracted features in order to homogenize the content. This includes:
- lowercase
- uniformization of null values

In [ ]:
# Convert the list of dict into a dataframe
output_image_features_df = pd.DataFrame(features_image)
output_image_features_df.reset_index(inplace=True, drop=True)
output_image_features_df.head()

In [22]:


# Convert all text columns to lower case  
for col in output_image_features_df.select_dtypes(include='object').columns:  
    output_image_features_df[col] = output_image_features_df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
# Replace occurrences of the string "None" with np.nan  
output_image_features_df.replace('none', np.nan, inplace=True)  

In [ ]:
output_image_features_df.head()

#### Export results to BQ

In [ ]:
# Export results to BQ
Dataframe(client, output_image_features_df).to_table(output_image_table)

#### Estimate costs

Cost estimation are base on the following references:
- [Google cloud pricing list](https://cloud.google.com/skus?hl=en&filter=gemini&currency=USD)
- [Vertex AI pricing list](https://cloud.google.com/vertex-ai/generative-ai/pricing)

In [ ]:
gcs_url = df_deduplicated_images.iloc[0]["image_url"]
bytes = image_to_bytes(gcs_url)

response = genai_client.models.generate_content(
    model='gemini-2.0-flash-exp', 
    contents=[
        'Describe the content of the picture',
        types.Part.from_bytes(data=bytes, mime_type="image/png")    
    ],
    config=types.GenerateContentConfig(
        temperature=0,
        response_mime_type= 'application/json',
        response_schema= ImageInformation
    )
)

In [ ]:
# Get the number of input tokens, output tokens, and cached tokens from the response  
input_tokens = response.usage_metadata.prompt_token_count  
output_tokens = response.usage_metadata.candidates_token_count  
intermediate_tokens = response.usage_metadata.cached_content_token_count  
  
# If there are cached tokens, print the count  
if intermediate_tokens:  
    print("Cached tokens:", intermediate_tokens)  
  
# Print the count of input tokens and output tokens  
print("Input tokens:", input_tokens)  
print("Output tokens:", output_tokens)  
  
# Get the number of images from the dataframe  
NUMBER_IMAGE = len(df_deduplicated_images)  
  
# Calculate the price based on the number of tokens  
price = input_tokens / 1E3 * 0.00001875 + output_tokens / 1E3 * 0.000075  
  
# Set the price per image  
image_price = 0.00002  
  
# Calculate the total price for all tokens and images  
total_price = (price + image_price) * NUMBER_IMAGE  
  
# Print the total price formatted as dollars with two decimal places  
print(f"Total estimated cost for {NUMBER_IMAGE} images: ${total_price}")  

### 4. Video Processing

#### Video Preprocessing
First we extract the size and aspect ratio of the video and send the information into an intermediary table.

In [ ]:
# Iterate over each row in the DataFrame, showing a progress bar  
for index, row in tqdm(df_test_video.iterrows(), total=df_test_video.shape[0]): 

    gcs_url = row["video_url"]  # Retrieve the image URL from the "image_url" column of the current row  
    vid = cv2.VideoCapture(gcs_url)
    height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
  
    if vid:  
        # If the image was successfully loaded, update the corresponding columns in the DataFrame  
        df_test_video.loc[index, "size"] = f"{height}:{width}"  # Store the image size in the "size" column  
        df_test_video.loc[index, "aspect_ratio"] = calculate_aspect(height, width)  # Calculate the aspect ratio and store it in the "aspect_ratio" column  
    else:  
        # If the image could not be loaded, update the corresponding columns with None values  
        df_test_video.loc[index, "size"] = None  
        df_test_video.loc[index, "aspect_ratio"] = None  

In [ ]:
# Check the content of a video
display(Video(df_test_video["video_url"].iloc[0]))

In [ ]:
df_test_video.reset_index(inplace=True, drop=True)
Dataframe(client, df_test_video).to_table(intermediate_video_table)

In [ ]:
class VideoInformation(BaseModel):  
    """Information about an image."""  
      
    image_description: str = Field(description="A brief one-sentence description of the creative.")  
    product_type: Literal["car", "service", "connected services", "accessories"] = Field(description="The type of product represented by the advertisement.")  
    contrast_presence: bool = Field(description="Is the contrast strong between text and background color? True if 'yes', False if 'no'.") 
    product_elements: List[str] = Field(description="List of short descriptions of products in the creative.") 
    meta_text: bool = Field(description="Is the text displayed in the first 5 seconds of the video? True if 'yes', False if 'no'.") 
    meta_video_length: bool = Field(description="Is the video duration more than 15 seconds ? True if 'yes', False if 'no'.") 
    meta_product_presence: bool = Field(description="Is the product (car) displayed in the first 3 seconds of the video? True if 'yes', False if 'no'.")  
    product_presence: bool = Field(description="Is the car displayed in the ad? True if 'yes', False if 'no'.")  
    product_share: int = Field(default = None, description="Percentage of video duration where the product is displayed")  
    product_percent: int = Field(description="Percentage of the creative area covered by the product.")  
    main_objects: List[str] = Field(description="List of the main objects in the picture.")  
    text: bool = Field(description="Indicates if there is text in the creative. True if 'yes', False if 'no'.")  
    text_location: Literal["top", "down", "right", "left", "center", "None"]= Field(default="None", description="Locations ad related text that is not the call to action")  
    brand_logo: bool = Field(description="Indicates if there is a brand logo on the product (car). True if 'yes', False if 'no'.")  
    promotion: bool = Field(description="Indicates if there is a promotion call in the creative. True if 'yes', False if 'no'.")  
    promotion_deadline: bool = Field(description="Indicates if a promotion deadline is displayed in the creative. True if 'yes', False if 'no'.")  
    promotion_theme: str= Field(default= "None", description="Specifies if the promotion is linked to a special event (e.g., Black Friday, Christmas).")  
    call_to_action: bool = Field(description="Indicates if there is a short call to action in the creative. The call to action should include a verb and it's length should have a maximum of 4 words. True if 'yes', False if 'no'.")  
    call_to_action_size: int = Field(default=None, description="Percentage of the creative area covered by the call to action, if applicable.")  
    car_brand: str = Field(default="None", description="The brand of the car, only if it is displayed on the product itself.") 
    car_model: str = Field(default="None", description="The model of the car, if applicable.")  
    price: bool = Field(description="Indicates if a price is displayed in the creative. True if 'yes', False if 'no'.")  
    ad_purpose: str = Field(default="awareness", description="The intended action promoted by the verb mentionned in the text (e.g., buy, order, discover, test).")  
    text_extraction: str = Field(default="None", description="The text content extracted from the creative.")  
    price_size: int = Field(default="None", description="Percentage of the creative area covered by the price, if applicable.")
    price_type: Literal["None", "net price", "recurring monthly"] = Field(default="None", description="The format in which the price is presented. This can either be a one-time full net price or a recurring monthly price.")   
    color_palette: List[str] = Field(description="Dominant colors used in the creative.")  
    human_elements: List[str] = Field(default="None", description="Short descriptions of main humans in the creative.")  
    human_presence: bool = Field(description="Are there any distinguishable humans displayed in the ad? True if 'yes', False if 'no'.") 
    human_number: int = Field(default="None", description="Number of distinguishable humans in the creative.")  
    human_genders: List[str] = Field(default="None", description="List of humans' gender in the picture.")  
    imagery_type: Literal["photographs", "illustration", "mix media", "infographics", "3D rendering", "icons"] = Field(description="Type of imagery in the creative.")  
    style: List[str] = Field(description="General aesthetic and style of the creative (e.g., modern, vintage, minimalist, abstract).")  
    actions: List[str] = Field(description="Activities or actions taking place in the creative (e.g., running, jumping, eating).")  
    actions_presence: bool = Field(description="Is there an action/story taking place in the ad? True if 'yes', False if 'no'.")  
    environment: List[Literal["indoor", "outdoor", "urban", "rural", "None"]] = Field(description="The environment in which the scene is set.")   
    time_of_day: Literal["not identifiable", "daytime", "nighttime", "sunset", "dawn"] = Field(description="The time of day in the creative.")  
    weather: Literal["None", "sunny", "snowy", "rainy"] = Field(default="None", description="The weather condition in the creative, if identifiable.")  
    background_main_color: str = Field(default="None", description="Dominant color of the background.")  
    human_shot_size: List[str] = Field(default="None", description="Shot size of the human in the creative (e.g., full shot, medium full shot, cowboy shot, closeup).")  
    human_age: Literal["senior", "adult", "child", "baby"] = Field(default="None", description="The age group of the human(s) in the  if they are distinguishable.")  
    electric_vehicle: bool = Field(description="Is the ad related to the electrification of the vehicle? True if 'yes', False if 'no'.")  
    goal_awareness_or_consideration_or_conversion: str = Field(default="None", description="The ad's purpose: awareness, consideration, or conversion.") 
    meta_logo_display: bool = Field(description="Is the brand referenced in the first 3 seconds of the video, without considering the logo on the product? True if 'yes', False if 'no'.")
    branding_share: int = Field(None, description="Time percentage of the creative where branding (logo, text, etc.) is shown.")  
    black_bars: bool = Field(description="Is there a black bar on the creative? True if 'yes', False if 'no'.")  
    text_length: int = Field(default = None, description="Number of lines the text takes on the creative without considering the call to action.") 
    promotion_display: bool = Field(description="Indicates whether a price discount, or price promotion message is displayed in the ad. True if 'yes', False if 'no'.")  
    call_to_action_verb: str = Field(default="None", description="The specific verb used in the call to action, if present (e.g., 'buy', 'test').")  
    promotion_wording_type: Literal["None", "gain", "loss_aversion"] = Field(default="None", description="Specifies the type of promotion wording used in the ad: 'gain' for emphasizing benefits, 'loss_aversion' for emphasizing avoidance of loss, or 'None' if there is no promotion wording.")  
    persuasion: Literal["None", "informative", "persuasive", "emotional", "humorous", "inspirational", "authoritative", "urgent", "relatable"] = Field(default="None", description="The tone of voice used for the advertisement, if applicable (e.g., Humour, Surprise, Suspense, Love, Happiness, Neutral).")  
    new_old_vehicle: Literal["None", "new", "second-hand"] = Field(description="Indicates whether the advertised vehicle is new or old.")  
    audio_presence: bool = Field(default="None", description="Is there audio (music, voice, ...) in the creative? True if 'yes', False if 'no'.")  
    subtitles_presence: bool = Field(None, description="Are there subtitles in the creative? True if 'yes', False if 'no'.")  
    product_usage: bool = Field(description="Is the product (car, connected feature, etc.) used in the Ad? Is there a creative of someone driving the car? True if 'yes', False if 'no'.")  
    action_audio: bool = Field(description="Is there a clear audio voice that is delivering the call to action? True if 'yes', False if 'no'.") 



features_video = []
video_analysis_prompt = """You are an expert video analyser, describe the video according the the following json schema. Don't take the subtitles of the video into account."""

def append_features_video():
    video_uri = row["video_url"]

    contents = [
        types.Part.from_uri(
            file_uri=video_uri,
            mime_type="video/*",
        ),
        video_analysis_prompt,
    ]

    response = genai_client.models.generate_content(
        model='gemini-2.0-flash-exp', 
        contents=contents,
        config=types.GenerateContentConfig(
            temperature=0,
            response_mime_type= 'application/json',
            response_schema= VideoInformation
        )
    )
    feature_dict = json.loads(response.text)
    feature_dict.update({"video_url": row["video_url"]})
    features_video.append(feature_dict)
    return response


for index, row  in tqdm(df_test_video.iterrows(), total=df_test_video.shape[0]):
    retry(append_features_video)


#### Estimate costs

In [ ]:
input_tokens = response.usage_metadata.prompt_token_count
output_tokens = response.usage_metadata.candidates_token_count
intermediate_tokens = response.usage_metadata.cached_content_token_count

if intermediate_tokens:
    print("Cached tokens: ", intermediate_tokens)

print("Input tokens: ", input_tokens)
print("Output tokens:", output_tokens)

NUMBER_VIDEO = len(df_test_video)
price = input_tokens/1E3*0.00001875 + output_tokens/1E3*0.000075
vide_price_per_sec = 0.00002 * 30 #estimation of 30 sec per video
print(f"Total estimated cost for {NUMBER_VIDEO}: ${(price + vide_price_per_sec) * NUMBER_VIDEO}" )

#### Features Post processing

In [ ]:
# Convert the list of dict into a dataframe
output_video_features_df = pd.DataFrame(features_video)
output_video_features_df.reset_index(inplace=True, drop=True)
output_video_features_df.head()

In [43]:
# Convert all text columns to lower case  
for col in output_video_features_df.select_dtypes(include='object').columns:  
    output_video_features_df[col] = output_video_features_df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [44]:
# Replace occurrences of the string "None" with np.nan  
output_video_features_df.replace('none', np.nan, inplace=True)  

In [ ]:
output_video_features_df.head()

#### Export to BQ

In [ ]:
Dataframe(client, output_video_features_df).to_table(output_video_table)

### 5. Reconstitution of final table

In [ ]:
from tools.octocloud import Query

sql_image = """ 

SELECT
  *
FROM(
  SELECT
    *EXCEPT(image_urls),
  FROM {image_inputs} LEFT JOIN UNNEST(image_urls) AS image_url 
  GROUP BY ALL
) AS main
INNER JOIN(
  SELECT
    *EXCEPT(id, category)
  FROM {image_intermediate}
)AS intermediate
USING(image_url)
INNER JOIN(
  SELECT
    *EXCEPT(image_url),
    COUNT(*) over (PARTITION BY image_hash) AS cnt
  FROM {image_outputs}
  ORDER BY cnt DESC
) AS image_features
USING(image_hash)


"""

query = sql_image.format(
    image_inputs=input_ads_table.path("standard"),
    image_intermediate=intermediate_image_table.path("standard"),
    image_outputs=output_image_table.path("standard")
)

job = Query(client, query).to_table(features_image_table)



In [ ]:
sql_video = """ 

SELECT
  *
FROM(
  SELECT
    *EXCEPT(image_urls, video_urls),
  FROM {video_inputs} LEFT JOIN UNNEST(video_urls) AS video_url
  GROUP BY ALL
) AS main
INNER JOIN(
  SELECT
    *EXCEPT(id, category)
  FROM {video_intermediate}
  GROUP BY ALL
)AS intermediate
USING(video_url)
INNER JOIN(
  SELECT
    *
  FROM {video_outputs}
) AS features
USING(video_url)


"""

query = sql_video.format(
    video_inputs=input_ads_table.path("standard"),
    video_intermediate=intermediate_video_table.path("standard"),
    video_outputs=output_video_table.path("standard")
)
# print(query)
job = Query(client, query).to_table(features_video_table)
